# Import Libraries

In [11]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [10]:

import numpy as np


import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("Tensorflow version : {}".format(tf.__version__))

## Import Dataset

In [ ]:

df_train = pd.read_csv('train.csv')


df_test = pd.read_csv('test.csv')

In [ ]:

train = df_train.drop(['SalePrice'], axis=1)
y = df_train['SalePrice']

In [ ]:

data=pd.concat([train, df_test], axis=0)

In [ ]:
# print each shape of data
print("The shape of data train : {} ".format(df_train.shape))
print("The shape of data test : {} ".format(df_test.shape))
print("The shape of all data (label excluded) : {} ".format(data.shape))

## Data Analysis

In [ ]:
data.describe()

In [ ]:

df_train.describe(include=['object', 'bool'])

In [ ]:
df_train.sort_values(by='SalePrice', ascending=False).head()

In [ ]:

df_analysis = {'Year Sold':['2006', '2007', '2008', '2009', '2010'],
               'Mean Sale Price': [round(df_train.query('YrSold == 2006').SalePrice.mean()),
                            round(df_train.query('YrSold == 2007').SalePrice.mean()),
                            round(df_train.query('YrSold == 2008').SalePrice.mean()),
                            round(df_train.query('YrSold == 2009').SalePrice.mean()),
                            round(df_train.query('YrSold == 2010').SalePrice.mean())],
               'Min Sale Price': [df_train.query('YrSold == 2006').SalePrice.min(),
                                  df_train.query('YrSold == 2007').SalePrice.min(),
                                  df_train.query('YrSold == 2008').SalePrice.min(),
                                  df_train.query('YrSold == 2009').SalePrice.min(),
                                  df_train.query('YrSold == 2010').SalePrice.min()],
               'Max Sale Price': [df_train.query('YrSold == 2006').SalePrice.max(),
                                  df_train.query('YrSold == 2007').SalePrice.max(),
                                  df_train.query('YrSold == 2008').SalePrice.max(),
                                  df_train.query('YrSold == 2009').SalePrice.max(),
                                  df_train.query('YrSold == 2010').SalePrice.max()]}
df_temp = pd.DataFrame(df_analysis, index=['1', '2', '3', '4', '5'])
df_temp

In [ ]:

sns.set(style='whitegrid')
sns.lineplot(x="Year Sold", y="Mean Sale Price", 
                  data=df_temp)

In [ ]:
for i in range(0, len(df_train.columns), 4):
    sns.pairplot(data=df_train,
                x_vars=df_train.columns[i:i+4],
                y_vars=['SalePrice'])

## Data Preparation

In [5]:
combined = data.copy()
nan_percentage = combined.isnull().sum().sort_values(
    ascending=False) / combined.shape[0]
missing_val = nan_percentage[nan_percentage > 0]

plt.figure(figsize=(9,7))
sns.barplot(x=missing_val.index.values, 
            y=missing_val.values * 100, 
            palette="Reds_r");
plt.title("Percentage of missing values in data");
plt.ylabel("%");
plt.xticks(rotation=90);

NameError: name 'data' is not defined

In [ ]:
data.drop(columns=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],
          axis=1, inplace=True)

In [ ]:
data.drop(columns=['Id'], inplace=True, axis=1)

In [ ]:
nan_col = data.columns[data.isnull().any()].tolist()
data[nan_col].head(5)

### Taking care NaN value with type is float

###### The strategy : replace with the average of that column

In [ ]:
df=data.fillna(data.mean())

In [ ]:
combined = df.copy()
nan_percentage = combined.isnull().sum().sort_values(
    ascending=False) / combined.shape[0]
missing_val = nan_percentage[nan_percentage > 0]

plt.figure(figsize=(9,7))
sns.barplot(x=missing_val.index.values, 
            y=missing_val.values * 100, 
            palette="Reds_r");
plt.title("Percentage of missing values in data");
plt.ylabel("%");
plt.xticks(rotation=90);

### Taking care NaN value with type is Object

###### The strategy : replace with mode of that column

In [ ]:
nan_col = df.columns[df.isnull().any()].tolist()

In [ ]:
for column in nan_col:
    df[column].fillna(df[column].mode()[0], inplace=True)

## Normalize the data

In [ ]:
numerical_columns=df.select_dtypes(exclude=['O']).columns

In [ ]:
df[numerical_columns].sample(5, random_state=99)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def normalize_data(df):
  numerical_columns=df.select_dtypes(exclude=['O']).columns

  scaling=MinMaxScaler()
  df[numerical_columns]=scaling.fit_transform(df[numerical_columns])
  return df

In [ ]:

df_norm = normalize_data(df)

In [ ]:

df_norm[numerical_columns].sample(5, random_state=99)

In [ ]:
df_norm.sample(5, random_state=99)

## Categorical Variable

In [ ]:
categorical_columns=df_norm.select_dtypes(include=['O']).columns

In [ ]:
print(categorical_columns)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import sklearn as s

In [ ]:

print(s.__version__)

In [ ]:

column_transformer = make_column_transformer(
    (OneHotEncoder(), categorical_columns),
    remainder='passthrough'
)

In [ ]:

df_encoded = column_transformer.fit_transform(df_norm)

In [ ]:
df_encoded

In [ ]:
# Change data type from sparse into DataFrame
import scipy.sparse
df_encode = pd.DataFrame.sparse.from_spmatrix(df_encoded)

In [ ]:
df_encode.sample(5, random_state=99)

# Split dataset

In [ ]:
X_train = df_encode.iloc[:-len(df_test), :]
X_test = df_encode.iloc[len(df_train):, :]

In [ ]:
print("\nOld data")
print("The shape of data train : {} ".format(df_train.shape))
print("The shape of data test : {} ".format(df_test.shape))

print("\nNew data")
print("The shape of data train : {} ".format(X_train.shape))
print("The shape of data test : {} ".format(X_test.shape))

print("The shape of class : {} ".format(y.shape))

## Training the model

In [ ]:

model = Sequential()


model.add(Dense(512, kernel_initializer='normal',
                   input_dim = X_train.shape[1], 
                   activation='relu'))

 
model.add(Dense(216, kernel_initializer='normal',activation='relu'))
model.add(Dense(216, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(64, kernel_initializer='normal',activation='relu'))
model.add(Dense(8, kernel_initializer='normal',activation='relu'))

 
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

model.compile(loss='mean_absolute_error', 
                 optimizer='adam', 
                 metrics=['mean_absolute_error'])
model.summary()

In [ ]:

filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'
callback = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode ='auto')

In [ ]:

model.fit(X_train, y, 
             epochs=1000, 
             batch_size=32, 
             validation_split = 0.25, 
             callbacks=[callback])

## Predict

In [ ]:

predictions = model.predict(X_test)

In [ ]:

submission = pd.read_csv('sample_submission.csv')
submission['SalePrice'] = predictions
submission.head(6)

In [ ]:
# Save the prediction
submission.to_csv('Submission.csv', index=False)